<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/pack/train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [4]:
!pip install git+https://github.com/lab-cosmo/librascal gpytorch pytorch-lightning wandb git+https://github.com/chem-gp/fande xtb nbstripout
! nbstripout --install --global

  Cloning https://github.com/lab-cosmo/librascal to /tmp/pip-req-build-j0sho39x
  Running command git clone --filter=blob:none --quiet https://github.com/lab-cosmo/librascal /tmp/pip-req-build-j0sho39x
  Resolved https://github.com/lab-cosmo/librascal to commit 6c55e99720f9a181f1efc5c7cd4976c87809d79b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/chem-gp/fande to /tmp/pip-req-build-2rd3t97a
  Running command git clone --filter=blob:none --quiet https://github.com/chem-gp/fande /tmp/pip-req-build-2rd3t97a
  Resolved https://github.com/chem-gp/fande to commit aded96b5f6a1eec9b699457a47248bcb58f290d1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.4 MB/s eta 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Cooking up the model

In [7]:
import os
# DATA_DIR = os.path.expanduser("~/repos/data/")
# # DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")


DATA_DIR = os.path.expanduser("/content/drive/MyDrive/data/")
# FANDE_DIR = os.path.expanduser("~/")
RESULTS_DIR = os.path.expanduser("/content/results")
# os.makedirs(RESULTS_DIR, exist_ok=True)


ENERGY_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
ENERGY_NUM_INDUCING_POINTS = 10
ENERGY_LR = 0.1
ENERGY_NUM_STEPS = 5

FORCES_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
FORCES_NUM_INDUCING_POINTS = 10
NUM_FORCE_SAMPLES = 10
FORCES_LR = 0.1
FORCES_NUM_STEPS = 5

PREDICTOR_NAME = 'test.pth'
SUBSAMPLE = 200 # subsample data to reduce time durings tests

! python cook_model.py \
--data_dir $DATA_DIR \
--results_dir $RESULTS_DIR \
--energy_model $ENERGY_MODEL \
--energy_num_inducing_points $ENERGY_NUM_INDUCING_POINTS \
--energy_lr $ENERGY_LR \
--energy_num_steps $ENERGY_NUM_STEPS \
--forces_model $FORCES_MODEL \
--forces_num_inducing_points $FORCES_NUM_INDUCING_POINTS \
--num_force_samples $NUM_FORCE_SAMPLES \
--forces_lr $FORCES_LR \
--forces_num_steps $FORCES_NUM_STEPS \
--predictor_name $PREDICTOR_NAME \
--subsample $SUBSAMPLE


DATA_DIR /content/drive/MyDrive/data/
RESULTS_DIR /content/results
ENERGY_MODEL variational_inducing_points
ENERGY_NUM_INDUCING_POINTS 10
ENERGY_LR 0.1
ENERGY_NUM_STEPS 5
FORCES_MODEL variational_inducing_points
FORCES_NUM_INDUCING_POINTS 10
NUM_FORCE_SAMPLES 10
FORCES_LR 0.1
FORCES_NUM_STEPS 5
PREDICTOR_NAME test.pth
5 5
fande.models module imported...
Icecream logger is not available
Total number of found groups: 14
Checking if all atoms are covered:  True
Total length of traj is 5
Total number of batches 5
Calculating invariants on trajectory with librascal...
100% 5/5 [00:00<00:00, 17.45it/s]
invariants for energy fitting calculated
100% 5/5 [00:10<00:00,  2.19s/it]
invariants for forces fitting calculated
torch.Size([5, 1200])
torch.Size([5])
Dataloader for group 0 created
Number of samples in dataloader: 10
Dataloader for group 1 created
Number of samples in dataloader: 10
Dataloader for group 2 created
Number of samples in dataloader: 10
Dataloader for group 3 created
Number of 

## Testing models

In [8]:
from ase import io
test_traj = io.read(DATA_DIR + "/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1000:1010")

In [9]:
import sys
import torch
FANDE_DIR = os.path.expanduser("~/repos/")
sys.path.append(FANDE_DIR + "fande/")

from fande.predict import FandePredictor
from fande.ase import FandeCalc
# load the predictor:
# predictor_loaded = torch.load(RESULTS_DIR + "/fande_predictor.pth")
predictor_loaded = torch.load(RESULTS_DIR + "/test.pth")
fande_calc_loaded = FandeCalc(predictor_loaded)
device = torch.device('cpu')
fande_calc_loaded.predictor.move_models_to_device(device)

INFO:fande:Fande module imported
No module named 'skmatter'


fande.models module imported...
Icecream logger is not available


In [10]:
forces_true = []
forces_predicted = []

energy_true = []
energy_predicted = []

for atoms in test_traj:
    forces_true.append(atoms.get_forces())
    energy_true.append(atoms.get_potential_energy())
    atoms.set_calculator(fande_calc_loaded)
    forces_predicted.append(atoms.get_forces())
    energy_predicted.append(atoms.get_potential_energy())

INFO:fande:Setting context for descriptors calculation to production
INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1783.3480834960938
Predicting forces...
Energy model summary: 
Time invariants:  0.15735626220703125
Time prediction:  4.910707473754883
Time moving on device:  0.7863044738769531
Time total:  7.155179977416992


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1619.107723236084
Predicting forces...
Energy model summary: 
Time invariants:  0.15354156494140625
Time prediction:  3.579854965209961
Time moving on device:  0.7348060607910156
Time total:  5.687952041625977


INFO:fande:Setting context for descriptors calculation to production


...

Time for invariants (call from forces):  1575.7570266723633
Predicting forces...
Energy model summary: 
Time invariants:  0.15735626220703125
Time prediction:  4.400014877319336
Time moving on device:  1.001119613647461
Time total:  7.058620452880859


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1569.098711013794
Predicting forces...
Energy model summary: 
Time invariants:  0.16736984252929688
Time prediction:  3.591299057006836
Time moving on device:  0.7672309875488281
Time total:  5.756139755249023


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1571.5913772583008
Predicting forces...
Energy model summary: 
Time invariants:  0.15687942504882812
Time prediction:  3.6416053771972656
Time moving on device:  0.7643699645996094
Time total:  5.811452865600586
Time for invariants (call from forces):  1578.7808895111084
Predicting forces...
Energy model summary: 
Time invariants:  0.15091896057128906
Time prediction:  3.762483596801758
Time moving on device:  0.8096694946289062
Time total:  6.184101104736328


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

energy_true = np.array(energy_true)
energy_predicted = np.array(energy_predicted)
energy_errors = energy_predicted - energy_true

forces_true = np.array(forces_true)
forces_predicted = np.array(forces_predicted)
forces_errors = forces_predicted - forces_true

atomic_groups = predictor_loaded.fdm.atomic_groups



for ag in atomic_groups:

    print("Atomic group", ag)
    print("F_x")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 0].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 0].flatten()**2)))
    print("F_y")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 1].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 1].flatten()**2)))
    print("F_z")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 2].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 2].flatten()**2)))
    print("E")
    print("MAE", np.mean(np.abs(energy_errors.flatten())))
    print("RMSE", np.sqrt(np.mean(energy_errors.flatten()**2)))

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
    axs[0].set_title(r"$F_x$")
    axs[0].plot(np.array(forces_true)[:, ag, 0].flatten(), label="true")
    axs[0].plot(np.array(forces_predicted)[:, ag, 0].flatten(), label="predicted")
    axs[0].legend()

    axs[1].set_title(r"$F_y$")
    axs[1].plot(np.array(forces_true)[:, ag, 1].flatten(), label="true")
    axs[1].plot(np.array(forces_predicted)[:, ag, 1].flatten(), label="predicted")
    axs[1].legend()

    axs[2].set_title(r"$F_z$")
    axs[2].plot(np.array(forces_true)[:, ag, 2].flatten(), label="true")
    axs[2].plot(np.array(forces_predicted)[:, ag, 2].flatten(), label="predicted")
    axs[2].legend()

    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

    axs[0].set_title(r"$F_x$ errors")
    axs[0].hist(forces_errors[:, ag, 0].flatten(), bins=100)

    axs[1].set_title(r"$F_y$ errors")
    axs[1].hist(forces_errors[:, ag, 1].flatten(), bins=100)

    axs[2].set_title(r"$F_z$ errors")
    axs[2].hist(forces_errors[:, ag, 2].flatten(), bins=100)
    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
    axs[0].set_title(r"$E$ errors")
    axs[0].hist(energy_errors.flatten(), bins=100)

    axs[1].set_title(r"$E$")
    axs[1].plot(energy_true, label="true")
    axs[1].plot(energy_predicted, label="predicted")
    axs[1].legend()
    plt.show()
